In [7]:
import os
import json
import cv2
import shutil

def convert_to_yolo(box, img_w, img_h):
    # box: [x, y, w, h] in pixel coordinates
    x, y, w, h = box
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return x_center, y_center, w_norm, h_norm

def piece_to_class(piece):
    # Map piece symbol to class index (customize as needed)
    mapping = {
        'K': 0, 'Q': 1, 'R': 2, 'B': 3, 'N': 4, 'P': 5,
        'k': 6, 'q': 7, 'r': 8, 'b': 9, 'n': 10, 'p': 11
    }
    return mapping.get(piece, -1)

def create_yolo_dataset(render_dir, out_dir):
    images_dir = os.path.join(out_dir, 'images')
    labels_dir = os.path.join(out_dir, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)
    for fname in os.listdir(render_dir):
        if not fname.endswith('.json'):
            continue
        json_path = os.path.join(render_dir, fname)
        img_name = fname.replace('.json', '.png')
        img_path = os.path.join(render_dir, img_name)
        label_path = os.path.join(labels_dir, fname.replace('.json', '.txt'))
        img_out_path = os.path.join(images_dir, img_name)

        with open(json_path, 'r') as f:
            data = json.load(f)
        img = cv2.imread(img_path)
        img_h, img_w = img.shape[:2]

        # Copy image to images folder
        shutil.copy2(img_path, img_out_path)

        with open(label_path, 'w') as f_out:
            for piece in data['pieces']:
                box = piece['box']
                if box is None:
                    continue
                class_id = piece_to_class(piece['piece'])
                if class_id == -1:
                    continue
                x_center, y_center, w_norm, h_norm = convert_to_yolo(box, img_w, img_h)
                f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

In [8]:
script_dir = os.getcwd()
render_path = os.path.join(script_dir, "..", "render")
output_path = os.path.join(script_dir, "..", "dataset")
create_yolo_dataset(render_path, output_path)
# Output will be in dataset/images and dataset/labels